# Global birth rate analysis

Birth rate by country and the global birth rate will be analyzed under the impacts of the following factors: 
- Financial: GDP per Capita
- Social: Happiness index, Social support score, life expectancy, freedom to make life choices, generosity,perception of corruption
- Jobs: Unemployment rate

In [187]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from scipy import stats
import statsmodels.api as sm

## Data Import & Data Preparation

In [188]:
# Data import
df_birth_rate = pd.read_csv("birth_rate.csv")
df_gdp = pd.read_csv("GDP.csv")
df_happiness_index = pd.read_excel("DataForFigure2.1WHR2023.xls")
df_unemployment_rate = pd.read_csv('unemployment_rate.csv')


In [189]:
# Drop duplicates & rename columns
df_birth_rate = df_birth_rate.rename({'country': 'Country'}, axis='columns')
df_gdp = df_gdp.drop_duplicates(subset='country', keep='first')
df_gdp = df_gdp.rename({'country': 'Country'}, axis='columns')
df_happiness_index = df_happiness_index.rename({'Country name': 'Country'},
                                               axis='columns')
df_unemployment_rate = df_unemployment_rate.rename({'country': 'Country'},
                                                    axis='columns')


In [190]:
# Preliminary variable selection
df_birth_rate = df_birth_rate[['birthRate', 'Country']]
df_gdp = df_gdp[['Country', 'gdpPerCapita']]
df_happiness_index = df_happiness_index[['Country', 'Ladder score', 'Social support', 
                                         'Healthy life expectancy',
                                         'Freedom to make life choices', 
                                         'Generosity', 'Perceptions of corruption']]
df_unemployment_rate = df_unemployment_rate[['Country', 'rateWb']]

In [191]:
# Main data frame preparation
df_final = df_birth_rate.merge(df_gdp,
                                on='Country').merge(df_happiness_index,
                                                     on='Country').merge(df_unemployment_rate, on='Country')
df_final.columns

Index(['birthRate', 'Country', 'gdpPerCapita', 'Ladder score',
       'Social support', 'Healthy life expectancy',
       'Freedom to make life choices', 'Generosity',
       'Perceptions of corruption', 'rateWb'],
      dtype='object')

In [192]:
df_final[['gdpPerCapita', 'Social support', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption']] = df_final[['gdpPerCapita', 'Social support', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption']]*100

In [193]:
df_final.loc[128,'rateWb'] = 3.56

In [194]:
# Data partition
df_predictors = df_final[['gdpPerCapita', 'Ladder score', 'Social support', 
                          'Healthy life expectancy', 'Freedom to make life choices',
                          'Generosity', 'Perceptions of corruption', 'rateWb']]
df_target = df_final['birthRate']

x_train, x_test, y_train, y_test = train_test_split(df_predictors,df_target,test_size=0.25)


## Model Fit & Model result

In [195]:
model = linear_model.LinearRegression()

In [196]:
model.fit(x_train,y_train)

LinearRegression()

In [197]:
# birth_rate fitted and birth_rate prediction
y_fitted = model.predict(x_train)
y_predict = model.predict(x_test)

In [198]:
# residuals from train & test
residuals_train = y_train - y_fitted
residuals_test = y_test - y_predict

In [199]:
# Coefficients
coef_gdp = model.coef_[0]
coef_happiness_index = model.coef_[1]
coef_social_support = model.coef_[2]
coef_life_expectancy = model.coef_[3]
coef_freedom = model.coef_[4]
coef_generosity = model.coef_[5]
coef_corruption = model.coef_[6]
coef_unemployment = model.coef_[7]
intercept = model.intercept_
model.coef_

array([ 2.77380576e-07, -1.34550587e+00, -1.76182416e-01, -1.14600048e+00,
       -5.37058613e-02,  3.39552033e-02, -6.26143088e-02, -7.50082220e-02])

In [200]:
print('model = %f + %f GDP %f Happiness index  %f Social support %f life expectancy  %f freedom  %f generosity  %f corruption  %f Unemployment' % (intercept, coef_gdp, coef_happiness_index, coef_social_support, coef_life_expectancy, coef_freedom, coef_generosity, coef_corruption, coef_unemployment))

model = 124.175674 + 0.000000 GDP -1.345506 Happiness index  -0.176182 Social support -1.146000 life expectancy  -0.053706 freedom  0.033955 generosity  -0.062614 corruption  -0.075008 Unemployment


In [201]:
stats_model = sm.OLS(y_train, x_train).fit()

## Model & Predictors Evaluation

In [202]:
def model_significance(y_train_set, y_pred, p):
    y_mean = np.mean(y_train_set)
    tss = ((y_train_set - y_mean)**2).sum()
    rss = ((y_train_set - y_pred)**2).sum()
    n = len(y_train_set)
    f_test = (tss-rss)/(rss/(n-p-1))
    print(f_test)
    if f_test > 1:
        print("Reject Ho. At least one of the model coefficients is significant.")
    if f_test <=1: 
        print("Accept Ho. No coefficients are useful.")


def coefficient_testing(coeff, residuals, trainset,p):
    """Perform Hypothesis testing to determine if Coefficient in regression is significant"""
    # Obtain the residual standard deviation
    residual_variance = np.var(residuals)
    x_mean = np.mean(trainset)
    x_squared = ((trainset - x_mean)**2).sum()
    coeff_var = residual_variance/ x_squared
    coeff_std = np.sqrt(coeff_var)
    print("coeff_var, std:",coeff_var, coeff_std)
    # Obtain t test
    print("Hypothesis: Ho: B1 = 0, Ha: B1 !=0")
    t_value = coeff/coeff_std
    print("t_value:", t_value)
    degree_of_freedom = len(trainset) - p -1
    p_value = stats.t.sf(abs(t_value), degree_of_freedom)*2
    print ("p_value: %.24f" % p_value)
    # Conclusion:
    if p_value < 0.05:
        print('Reject Ho. The coefficient is significant, and has a relationship with the respond variable')
    if p_value > 0.05: 
        print('Accept Ho. The coefficient is not significant, and does not have a relationship with respond variable')
    return None


def residual_zero_mean(residuals,p):
    n = len(residuals)
    residual_mean = np.mean(residuals)
    residual_std = np.std(residuals)
    t_test = (residual_mean)/(residual_std/np.sqrt(n))
    p_value = stats.t.sf(t_test, n - p - 1)
    if p_value > 0.05:
        print("As the p_value is larger than 0.05, there is no sufficient evidence to reject Ho. From this, residuals mean is equal to 0. Linear regression assumption on residuals is appropriate.")
    elif p_value <= 0.05: 
        print("As p_value is smaller than 0.95, there is sufficient evidence to reject Ho. Thus zero mean assumption is not appropriate.")

def tss_rss(y_train, y_pred):
    y_mean = np.mean(y_train)
    tss = ((y_train - y_mean)**2).sum()
    rss = ((y_train - y_pred)**2).sum()
    n = len(y_train)
    return tss, rss

### Model Evaluation

In [203]:
# Model evaluation - F test
model_significance(y_train, y_fitted, 8)

374.4449859320317
Reject Ho. At least one of the model coefficients is significant.


### Coefficient evaluation


In [205]:
# GDP coefficient
coefficient_testing(coef_gdp, residuals_train, x_train['gdpPerCapita'],8)

coeff_var, std: 2.978640166121947e-14 1.7258737399131917e-07
Hypothesis: Ho: B1 = 0, Ha: B1 !=0
t_value: 1.6071892721486665
p_value: 0.111555021055602929247463
Accept Ho. The coefficient is not significant, and does not have a relationship with respond variable


In [206]:
# Happiness index
coefficient_testing(coef_happiness_index, residuals_train, x_train['Ladder score'], 8)

coeff_var, std: 0.1836779433674195 0.4285766481825853
Hypothesis: Ho: B1 = 0, Ha: B1 !=0
t_value: -3.1394754537209786
p_value: 0.002296736795010768765468
Reject Ho. The coefficient is significant, and has a relationship with the respond variable


In [207]:
# Social support
coefficient_testing(coef_social_support, residuals_train, x_train['Social support'],8)

coeff_var, std: 0.001425189779032586 0.0377516857773608
Hypothesis: Ho: B1 = 0, Ha: B1 !=0
t_value: -4.66687546393868
p_value: 0.000010736955975117387687
Reject Ho. The coefficient is significant, and has a relationship with the respond variable


In [208]:
# Healthy life expectancy
coefficient_testing(coef_life_expectancy, residuals_train, x_train['Healthy life expectancy'], 8)

coeff_var, std: 0.005876228984458393 0.07665656517519157
Hypothesis: Ho: B1 = 0, Ha: B1 !=0
t_value: -14.949802116037349
p_value: 0.000000000000000000000000
Reject Ho. The coefficient is significant, and has a relationship with the respond variable


In [209]:
# Freedom to make life choices
coefficient_testing(coef_freedom, residuals_train, x_train['Freedom to make life choices'], 8)


coeff_var, std: 0.0024895155018705177 0.049895044862897135
Hypothesis: Ho: B1 = 0, Ha: B1 !=0
t_value: -1.0763766521607552
p_value: 0.284667966006711514737049
Accept Ho. The coefficient is not significant, and does not have a relationship with respond variable


In [210]:
# Generosity
coefficient_testing(coef_generosity, residuals_train, x_train['Generosity'], 8)

coeff_var, std: 0.0011360126230705568 0.03370478635254282
Hypothesis: Ho: B1 = 0, Ha: B1 !=0
t_value: 1.0074297154272736
p_value: 0.316459468396020848768302
Accept Ho. The coefficient is not significant, and does not have a relationship with respond variable


In [211]:
# Perceptions of corruption
coefficient_testing(coef_corruption, residuals_train, x_train['Perceptions of corruption'], 8)

coeff_var, std: 0.0006541790977310535 0.02557692510312867
Hypothesis: Ho: B1 = 0, Ha: B1 !=0
t_value: -2.448078045418914
p_value: 0.016324182848884007807078
Reject Ho. The coefficient is significant, and has a relationship with the respond variable


In [212]:
# Unemployment rate: rateWb
coefficient_testing(coef_unemployment, residuals_train, x_train['rateWb'], 8)

coeff_var, std: 0.006887599201869825 0.0829915610280336
Hypothesis: Ho: B1 = 0, Ha: B1 !=0
t_value: -0.9038054116180471
p_value: 0.368538482718650750591394
Accept Ho. The coefficient is not significant, and does not have a relationship with respond variable


## Accuracy Evaluation

#### Train set

In [213]:
tss_train, rss_train = tss_rss(y_train, y_fitted)

In [216]:
n_train = len(y_train)
rse_train = np.sqrt(rss_train/(n_train-8-1))
r_squared_train = (tss_train-rss_train)/tss_train

In [217]:
print("RSE:", rse_train)
print("R-squared:", r_squared_train)

RSE: 4.719022710470406
R-squared: 0.8079599462684605


#### Test set

In [218]:
tss_test, rss_test = tss_rss(y_test, y_predict)
n_test = len(y_test)
rse_test = np.sqrt(rss_test/(n_test - 8-1))
r_squared_test = (tss_test-rss_test)/tss_test

In [219]:
print("RSE:", rse_test)
print("R-squared:", r_squared_test)

RSE: 6.42302184924035
R-squared: 0.6070800115534362


## Residuals Evaluation

Residuals will be assessed for the following factors: 
- Mean of zero
- Normally distributed
- Constant variance
- No correlation